# Graph Coloring

In [1]:
from datetime import datetime

def LogInfo(msg):
    print(datetime.now().strftime('%H:%M:%S') + ' - ' + msg)

def ReadFile(file_location):
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
    return input_data

def GetInputs(input_data):
    input_data = input_data.splitlines()
    input_data = [list(map(int, x.split(' ')))  for x in input_data]
    inputs={}
    inputs['nbnodes'] = input_data[0][0]
    inputs['nbedges'] = input_data[0][1]
    inputs['startnodes'] = [row[0] for row in input_data[1:]]
    inputs['endnodes'] = [row[1] for row in input_data[1:]]
    adjacentnodes = {}
    for row in input_data[1:]:
        node1 = row[0]
        node2 = row[1]
        if node1 not in adjacentnodes:
            adjacentnodes[node1] = []
        if node2 not in adjacentnodes[node1]:
            adjacentnodes[node1].append(node2)
        if node2 not in adjacentnodes:
            adjacentnodes[node2] = []
        if node1 not in adjacentnodes[node2]:
            adjacentnodes[node2].append(node1)
    for k, v in adjacentnodes.items():
        v.sort()
    inputs['nodes'] = list(adjacentnodes.keys())
    inputs['colors'] = range(len(inputs['nodes'])) # initialize colors to one for each node
    inputs['adjacentnodes'] = adjacentnodes
    return inputs

inputs = GetInputs(ReadFile('data/gc_20_1'))

### Option 1: Greedy algorithm

In [38]:
# function to run the algorithm for a given, ordered, list of nodes 
def RunGreedyAlgorithm(inputNodes, inputs):
    nodecolors = {}

    # loop on each node
    for node in inputs['nodes']:
        #print('>> Node ', node)
        for color in inputs['colors']:
            #print('    >> Color ', color)
            # try to assign each color until one is valid
            # to determine if a color is valid, we look at all adjacent nodes and discard already assigned colors.
            isAvailable = True

            if node not in inputs['adjacentnodes']:
                # if the node doesn't have any adjacent nodes, assign the first available color
                print('does not have any neighboors')
                nodecolors[node] = color
                break

            adjacentNodes = inputs['adjacentnodes'][node]
            for adjacentNode in adjacentNodes:
                if adjacentNode in nodecolors:
                    # if the node already has a color assigned ...
                    if nodecolors[adjacentNode] == color:
                        # ... and if the color assigned is the same than the current one, discard it.
                        isAvailable = False
                        break

            if isAvailable:
                nodecolors[node] = color
                break

    output = {}
    output['objective'] = len(set(nodecolors.values()))
    output['variables'] = list(nodecolors.values())
    return output

### Option 2: OR Tools - CP

In [39]:
from ortools.constraint_solver import pywrapcp

# Creates the solver
solver = pywrapcp.Solver("GraphColoring")

# run greedy algorithm first, to find an upper bound on the number of required colors
greedyoutputs = RunGreedyAlgorithm(inputs['nodes'], inputs)
print('Greedy objective: ', greedyoutputs['objective'])

# Creates the variables
variables = []
for i in range(inputs['nbnodes']):
    # x = solver.IntVar(0, inputs['nbnodes'] - 1, "x" + str(i))
    x = solver.IntVar(0, greedyoutputs['objective'] - 1, "x" + str(i))
    variables.append(x)

# Create the constraints
for edge in range(inputs['nbedges']):
    # get variables corresponding to start and end nodes
    snode = inputs['startnodes'][edge]
    enode = inputs['endnodes'][edge]
    #print('Adding constraint: ', snode, ' != ', enode)
    xs = variables[snode]
    xe = variables[enode]
    solver.Add(xs != xe)
    
# Create the decision builder
db = solver.Phase(variables, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

# Start solving
starttime = datetime.now()
solver.Solve(db)

count = 0
outputs = {}
while solver.NextSolution():
    count += 1
    # get variable values
    results = []
    for x in variables:
        results.append(x.Value())
    objective = len(set(results))
    if outputs == {}:
        outputs['objective'] = objective
        outputs['variables'] = results
    elif objective < outputs['objective']:
        outputs['objective'] = objective
        outputs['variables'] = results
    if (count % 10000) == 0:
        LogInfo("Solution " + str(count) + '; objective=' + str(outputs['objective']))

print('Run took ' + str(datetime.now() - starttime))
print(count, ' solutions scanned.')
print('objective: ', outputs['objective'])
print('variables: ', outputs['variables'])

Greedy objective:  3
16:59:03 - Solution 10000; objective=3
16:59:04 - Solution 20000; objective=3
16:59:04 - Solution 30000; objective=3
16:59:04 - Solution 40000; objective=3
16:59:04 - Solution 50000; objective=3
16:59:04 - Solution 60000; objective=3
16:59:04 - Solution 70000; objective=3
16:59:04 - Solution 80000; objective=3
16:59:04 - Solution 90000; objective=3
16:59:04 - Solution 100000; objective=3
16:59:04 - Solution 110000; objective=3
16:59:04 - Solution 120000; objective=3
16:59:04 - Solution 130000; objective=3
16:59:05 - Solution 140000; objective=3
16:59:05 - Solution 150000; objective=3
16:59:05 - Solution 160000; objective=3
16:59:05 - Solution 170000; objective=3
16:59:05 - Solution 180000; objective=3
16:59:05 - Solution 190000; objective=3
16:59:05 - Solution 200000; objective=3
16:59:05 - Solution 210000; objective=3
16:59:05 - Solution 220000; objective=3
16:59:05 - Solution 230000; objective=3
16:59:05 - Solution 240000; objective=3
Run took 0:00:01.980961
2488

### Option 3: Relaxation + branch and bounds

In [2]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('GraphColoringSolver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Objective function: minimize y (cap)
objective = solver.Objective()
objective.SetMinimization()

# Constraints
# ************************************************************

# constraint 1): cap
# Add an artifical cap variable, y, that will be superior to the maximum color of the graph.
# Minimizing y will squash the max color to its minimal possible value, which is 
# equivalent to finding the optimal graph coloring. 
constraint_cap = solver.Constraint(0, solver.infinity()) 

# Variables
y = solver.IntVar(0.0, inputs['nbnodes'], 'y')
constraint_cap.SetCoefficient(y, 1)
objective.SetCoefficient(y, 1)

variables = []
for n in range(inputs['nbnodes']):
    # constraint 2): each node must be assigned a unique color
    constraint_unique_node_color = solver.Constraint(1, 1)
    variables_colors = []
    for c in inputs['colors']:
        # add one variable for each {node+color} pairs
        x = solver.BoolVar('x_n' + str(n) + '_c' + str(c))
        variables_colors.append(x)
        # add to constraint 1)
        constraint_cap.SetCoefficient(x, (-1)*c)
        # add to constraint 2)
        constraint_unique_node_color.SetCoefficient(x, 1)
    variables.append(variables_colors)

# constraint 3): adjacent nodes cannot have the same color
for e in range(inputs['nbedges']):
    for c in inputs['colors']:
        #print('Add constraint: color ', c, ': ', inputs['startnodes'][e], ' != ', inputs['endnodes'][e])
        constraint_adj_nodes = solver.Constraint(0, 1) 
        constraint_adj_nodes.SetCoefficient(variables[inputs['startnodes'][e]][c], 1)
        constraint_adj_nodes.SetCoefficient(variables[inputs['endnodes'][e]][c], 1)
    
# Solve
LogInfo('Start solving...')
solver.Solve()
LogInfo('Solver finished.')
    
results = []
for n in range(inputs['nbnodes']):
    results.append(-1)
    for c in inputs['colors']:
        # print('node ', n, ', color ', c, ': ', )
        if 1 == variables[n][c].solution_value():
            results[n] = int(c)
            break
# objectiveValue = objective.Value()
objectiveValue = max(results)

# Results
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('x = ', results)
print('Optimal objective value =', objectiveValue)

outputs = {}
outputs['objective'] = int(objectiveValue)
outputs['variables'] = results

17:12:47 - Start solving...
17:12:47 - Solver finished.
Number of variables = 401
Number of constraints = 481
x =  [0, 2, 0, 0, 0, 0, 1, 1, 0, 1, 0, 2, 0, 1, 1, 0, 1, 1, 0, 0]
Optimal objective value = 2
